In [ ]:
!pip install timm
!pip install foolbox
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/ViT
# %pwd
# %rm -rf TinyImageNet/
# !unzip TinyImageNet.zip

# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
# !unzip tiny-imagenet-200.zip
# !rm tiny-imagenet-200.zip


     |████████████████████████████████| 296kB 6.6MB/s 
     |████████████████████████████████| 1.7MB 5.2MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 163kB 14.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Mounted at /content/drive
/content/drive/My Drive/ViT


In [ ]:
import timm
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform


from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM
import foolbox.attacks as fa

import os
import copy
import time 
import matplotlib.pyplot as plt
import numpy as np

import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from pathlib import Path
from tqdm import tqdm





In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/content/drive/MyDrive/598dataset/Cifar10/"

# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 2

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

input_size = 224


print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

CUDA Available:  True


In [ ]:
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
kwargs = {'num_workers': 1, 'pin_memory': True}

trainset = datasets.CIFAR10(root=data_dir, train=True,
                                        download=True, transform=transform['train'])
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, **kwargs)

testset = datasets.CIFAR10(root=data_dir, train=False,
                                       download=True, transform=transform['val'])
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, **kwargs)
dataloaders_dict = {'train': trainloader, 'val': testloader}

class_names = trainset.classes

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('')
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            with tqdm(dataloaders[phase], unit='batch') as tepoch:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tepoch:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        # Get model outputs and calculate loss
                        # Special case for inception because in training it has an auxiliary output. In train
                        #   mode we calculate the loss by summing the final output and the auxiliary output
                        #   but in testing we only consider the final output.
                        if is_inception and phase == 'train':
                            # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                            outputs, aux_outputs = model(inputs)
                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4*loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                if phase == 'val':
                    val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# save image to folders
def save_adv_images(inputs, clipped_advs, labels, model, dirpath=""):

  dir = dirpath
  if len(dirpath) == "":
    timestr = time.strftime("%Y%m%d-%H%M%S")
    dirpath = os.path.join("/content/drive/MyDrive/598dataset", timestr)

  
  # try to create the dir
  os.makedirs(dir, exist_ok=True)

  # loop over images and save adv results
  clean_preds = model(inputs).argmax(axis=1)
  adv_preds = model(clipped_advs).argmax(axis=1)
  norm = ((inputs - clipped_advs)**2).sum(axis=(1, 2, 3))**0.5
  norm = norm.cpu().numpy().round(2)

  # remember dir count
  filenum_counter = {}

  for i in range(inputs.shape[0]):

    adv_img = clipped_advs[i].cpu().numpy()
    adv_img = adv_img.swapaxes(0, 1)
    adv_img = adv_img.swapaxes(1, 2)

    # get labels and predictions
    label = class_names[int(labels[i])]
    clean_pred = class_names[int(clean_preds[i])]
    adv_pred = class_names[int(adv_preds[i])]
    img_dir = os.path.join(dir, str(label))

    if img_dir not in filenum_counter:
      os.makedirs(img_dir, exist_ok=True)
      filenum_counter[img_dir] = len(os.listdir(img_dir)) + 1
    else:
      filenum_counter[img_dir] += 1

    # image name
    img_name = "img{}_clean{}_adv{}_L2diff{}.png".format(filenum_counter[img_dir], clean_pred, adv_pred, norm[i])
    img_path = os.path.join(img_dir, img_name)

    # save the image
    plt.imsave(img_path, adv_img)

def test_attack(attack, epsilons, fmodel, dataloaders, img_save_dir="", save_train=False):
  count = 0
  running_acc = torch.zeros(len(epsilons)).to(device)

  # Vlidation set
  with tqdm(dataloaders['val'], unit='batch') as tepoch:
    for inputs, labels in tepoch:
      inputs = inputs.to(device)
      labels = labels.to(device)
      raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)

      # save images
      if img_save_dir:
        for idx in range(len(clipped_advs)):
          dirname = "Epsilon{}".format(epsilons[idx]) + "/val"
          save_adv_images(inputs, clipped_advs[idx], labels, fmodel, dirpath=os.path.join(img_save_dir, dirname))

      running_acc += 1 - success.float().mean(axis=-1)
      count += 1

  # report accuracy on val dataset
  running_acc /= count
  for eps, acc in zip(epsilons, running_acc):
      print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
  
  # save train adv
  if save_train:
    with tqdm(dataloaders['train'], unit='batch') as tepoch:
      for inputs, labels in tepoch:
        inputs = inputs.to(device)
        labels = labels.to(device)
        raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)

        # save images
        if img_save_dir:
          for idx in range(len(clipped_advs)):
            dirname = "Epsilon{}".format(epsilons[idx]) + "/train"
            save_adv_images(inputs, clipped_advs[idx], labels, fmodel, dirpath=os.path.join(img_save_dir, dirname))

In [ ]:



# Freeze model and replace last layer with CiFar10 num class output 
model = timm.create_model('vit_base_patch16_224', pretrained=True)
set_parameter_requires_grad(model, feature_extract)
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)
model.eval()
print(model)


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (

## ViT Attack


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ViTbase_16_224_cifar10.pth'))
preprocessing = dict(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

count = 0
running_acc = 0
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    running_acc += accuracy(fmodel, inputs, labels)
    count += 1
print(f"clean accuracy:  {running_acc/count * 100:.1f} %")

100%|██████████| 313/313 [01:00<00:00,  5.17batch/s]

clean accuracy:  93.0 %


In [ ]:
attack = fa.LinfFastGradientAttack()
epsilons_list = [
    0.001, 
    0.005,
    0.01, 
    0.1
]
for epsilons in epsilons_list: 
  epsilons = [epsilons]
  count = 0
  running_acc = torch.zeros(len(epsilons)).to(device)
  with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
    for inputs, labels in tepoch:
      inputs = inputs.to(device)
      labels = labels.to(device)
      # generate a list of adversarial for each input 
      raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
      running_acc += 1 - success.float().mean(axis=-1)
      count += 1
  running_acc /= count
  print(count)
  for eps, acc in zip(epsilons, running_acc):
      print(f"FGSM {epsilons} attack accuracy:  {acc.item() * 100:4.1f} %")


# img_save_dir = '/content/drive/MyDrive/598dataset/Cifar10/ViT/FGSM'
# test_attack(attack, epsilons, fmodel_cnn, dataloaders_dict, img_save_dir, save_train=True)

100%|██████████| 313/313 [03:05<00:00,  1.69batch/s]


313


  0%|          | 0/313 [00:00<?, ?batch/s]

FGSM [0.001] attack accuracy:  61.0 %


100%|██████████| 313/313 [03:05<00:00,  1.69batch/s]


...

313


  0%|          | 0/313 [00:00<?, ?batch/s]

FGSM [0.01] attack accuracy:  25.4 %


100%|██████████| 313/313 [03:05<00:00,  1.69batch/s]


313
FGSM [0.1] attack accuracy:  13.3 %


In [ ]:
attack = fa.LinfPGD()
epsilons = [
    0.001, 
    0.005,
    0.01, 
    0.1
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [1:24:47<00:00, 16.26s/batch]


  Linf norm ≤ 0.001 : 47.9 %


In [ ]:
attack = fa.L2PGD()
epsilons = [
    0.1,
    0.3,
    0.5, 
    2
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [1:00:03<00:00, 11.51s/batch]

  Linf norm ≤ 0.3   : 35.9 %


In [ ]:
attack = fa.L2CarliniWagnerAttack(steps=30, stepsize=0.0015)
epsilons = [
    0.1,
    0.3
]
count = 0
running_acc = torch.zeros(len(epsilons))
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1).to('cpu')
    count += 1
    
    torch.cuda.empty_cache()

running_acc /= count
for eps, acc in zip(epsilons, running_acc):
  print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [7:16:06<00:00, 83.60s/batch]

  L2 norm ≤ 0.1   : 78.4 %
  L2 norm ≤ 0.3   : 13.7 %


In [ ]:
attack = fa.L2CarliniWagnerAttack(steps=30, stepsize=0.0015)
epsilons = [
    0.5,
    2
]
count = 0
running_acc = torch.zeros(len(epsilons))
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1).to('cpu')
    count += 1
    
    torch.cuda.empty_cache()

running_acc /= count
for eps, acc in zip(epsilons, running_acc):
  print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [7:15:39<00:00, 83.51s/batch]

  L2 norm ≤ 0.5   :  2.6 %
  L2 norm ≤ 2     :  0.0 %


##FineTuning ResNet50



In [ ]:
model_cnn = models.resnet50(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

# set_parameter_requires_grad(model_cnn, feature_extract)

model_cnn.fc = nn.Linear(model_cnn.fc.in_features, num_classes)
model_cnn = model_cnn.to(device)
params_to_update = model_cnn.parameters()


print("Params to learn:")

for name,param in model_cnn.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

# # Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_cnn.parameters(), lr=0.001, momentum=0.9)
# # Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_cnn, hist = train_model(model_cnn, dataloaders_dict, criterion, optimizer_ft, num_epochs=2, is_inception=False)
torch.save(model_cnn.state_dict(), '/content/drive/MyDrive/598dataset/ResNet50_cifar10.pth')



##ResNet50 Attack

In [ ]:
model_cnn = models.resnet50(pretrained=True, progress=True)
model_cnn.fc = nn.Linear(model_cnn.fc.in_features, num_classes)
model_cnn = model_cnn.to(device)
model_cnn.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ResNet50_cifar10.pth'))
model_cnn.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
preprocessing = dict(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel_cnn = PyTorchModel(model_cnn, bounds=(0, 1), preprocessing=preprocessing)

count = 0
running_acc = 0
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    running_acc += accuracy(fmodel_cnn, inputs, labels)
    count += 1

print(f"clean accuracy:  {running_acc/count * 100:.1f} %")

100%|██████████| 313/313 [00:17<00:00, 17.66batch/s]

clean accuracy:  94.4 %


In [ ]:
# !rm -rf /content/drive/MyDrive/598dataset/Cifar10/ResNet50/FGSM/
attack = fa.LinfFastGradientAttack()
epsilons = [
    0.001, 
    0.005,
    0.01, 
    0.1
]

count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel_cnn, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"FGSM {epsilons} attack accuracy:  {acc.item() * 100:4.1f} %")

# img_save_dir = '/content/drive/MyDrive/598dataset/Cifar10/ResNet50/FGSM'
# test_attack(attack, epsilons, fmodel_cnn, dataloaders_dict, img_save_dir, save_train=True)

100%|██████████| 313/313 [04:17<00:00,  1.21batch/s]

FGSM [0.001, 0.005, 0.01, 0.1] attack accuracy:  63.5 %
FGSM [0.001, 0.005, 0.01, 0.1] attack accuracy:  21.4 %
FGSM [0.001, 0.005, 0.01, 0.1] attack accuracy:  15.0 %
FGSM [0.001, 0.005, 0.01, 0.1] attack accuracy:  19.8 %


In [ ]:
attack = fa.LinfPGD()
epsilons = [
    0.001, 
    0.005,
    0.01, 
    0.1
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel_cnn, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [2:12:12<00:00, 25.34s/batch]

  Linf norm ≤ 0.001 : 52.9 %
  Linf norm ≤ 0.005 :  0.2 %
  Linf norm ≤ 0.01  :  0.0 %
  Linf norm ≤ 0.1   :  0.0 %


In [ ]:
attack = fa.L2PGD()
epsilons = [
    0.1, 
    0.3,
    0.5,
    2
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel_cnn, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

# img_save_dir = '/content/drive/MyDrive/598dataset/Cifar10/ResNet50/L2PGD'
# test_attack(attack, epsilons, fmodel_cnn, dataloaders_dict, img_save_dir, save_train=True)

100%|██████████| 313/313 [2:45:23<00:00, 31.70s/batch]

  L2 norm ≤ 0.1   : 82.1 %
  L2 norm ≤ 0.3   : 41.7 %
  L2 norm ≤ 0.5   : 14.9 %
  L2 norm ≤ 2     :  0.0 %


In [ ]:
attack = fa.L2CarliniWagnerAttack(steps=30, stepsize=0.0015)
epsilons = [
    0.1, 
    0.3,
    0.5,
    2
]
count = 0
running_acc = torch.zeros(len(epsilons))
with tqdm(dataloaders_dict['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel_cnn, inputs, labels, epsilons=epsilons)
    running_acc += 1 - success.float().mean(axis=-1).to('cpu')
    count += 1
    
    torch.cuda.empty_cache()

running_acc /= count
for eps, acc in zip(epsilons, running_acc):
  print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 313/313 [3:05:30<00:00, 35.56s/batch]

  L2 norm ≤ 0.1   : 85.9 %
  L2 norm ≤ 0.3   : 26.0 %
  L2 norm ≤ 0.5   :  4.8 %
  L2 norm ≤ 2     :  0.0 %
